In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
Data_dir = '/content/gdrive/My Drive/Colab Notebooks/Datasets/MahineHack - Retail Price Prediction/'
df = pd.read_csv(Data_dir+ 'sample_Data.csv')
df.head()

,index,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Year,Month,Day,Timelight,Weekday
0,63388,0,1698,11,0.85,13246.0,35,2011,A,week2,Noon,Thursday
1,257072,0,1698,2,0.85,17337.0,35,2011,A,week1,Noon,Sunday
2,97281,0,1698,12,0.85,12451.0,32,2011,A,week2,Morning,Wednesday
3,153534,0,1698,60,0.85,14258.0,35,2011,A,week1,Noon,Thursday
4,255084,1,1536,2,0.39,17870.0,35,2011,C,week3,Morning,Tuesday


In [ ]:
df1 = df.copy()
df1.drop(['index'],axis=1,inplace=True)

In [ ]:
df1.corr()['UnitPrice'].sort_values(ascending=False)

UnitPrice      1.000000
StockCode      0.046119
Year          -0.000809
Quantity      -0.002372
Description   -0.002503
CustomerID    -0.018407
Country       -0.028692
Name: UnitPrice, dtype: float64

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


num_attribs = ['StockCode','Quantity','Description','CustomerID']
cat_attribs = [ 'Country','Weekday','Timelight','Year','Month','Day']

full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

In [ ]:
X= df1[['StockCode', 'Quantity', 'Country','Year','Month','Day','Weekday','Timelight','Description','CustomerID']]
y = df1['UnitPrice']

#X2 = df2[['StockCode', 'Quantity', 'Country','Year','Month','Day','Weekday','Timelight','Description','CustomerID']]
#y2 = df2['UnitPrice']

In [ ]:
X = full_pipeline.fit_transform(X)
#X2 = full_pipeline.fit_transform(X2)
print(X.shape)
#print(X2.shape)

(39968, 58)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=None,shuffle=True)

#from sklearn.model_selection import train_test_split
#X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2, test_size=0.25, random_state=None)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

rfr1 = RandomForestRegressor(max_depth=20, random_state=23)
rfr2 = RandomForestRegressor(max_depth=20, random_state=23)

rfr1.fit(X_train1,y_train1)
rfr2.fit(X_train2,y_train2)

pred_rfr1 = rfr1.predict(X_test1)
pred_rfr2 = rfr2.predict(X_test2)

print('The error in Random forest Regression df1 is: '+ str(np.sqrt(mean_squared_error(y_test1, pred_rfr1))))
print('The error in Random forest Regression df2 is: '+ str(np.sqrt(mean_squared_error(y_test2, pred_rfr2))))

The error in Random forest Regression df1 is: 18.669969637031727
The error in Random forest Regression df2 is: 43.93437693608586


#####Country grouping is not effective.
####Let`s try other models and hypertuning on df1 

Random Forest Regressor

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from pprint import pprint

rf = RandomForestRegressor()

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

# Randomized Search CV

# Number of trees in random forest
n_estimators = randint(low=1, high=250)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(0, 30, num = 10)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap' : bootstrap}
          

pprint(random_grid)

rf = RandomForestRegressor()

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}
{'bootstrap': [True, False],
 'max_depth': [0, 3, 6, 10, 13, 16, 20, 23, 26, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 15, 100],
 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f465ba343c8>}


In [ ]:
from sklearn.model_selection import train_test_split

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1,scoring='neg_mean_squared_error', random_state=42)

# Fit the random search model
# In order to test these models I will need to do a train test split with the training data-set. 
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 45.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 86.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [ ]:
rf_random.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=23, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=81, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
feature_importances = rf_random.best_estimator_.feature_importances_

In [ ]:
import collections
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for i in range(0,6):
  cat_one_hot_attribs.append(list(cat_encoder.categories_[i]))

def flatten(l):
   for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            for sub in flatten(el):
                yield sub
        else:
            yield el


cat_one_hot_attribs = list(flatten(cat_one_hot_attribs))

attributes = num_attribs + cat_one_hot_attribs
attributes = [str(i) for i in attributes]
sorted(zip(feature_importances, attributes), reverse=True)

[(0.6910699148149979, '35'),
 (0.28374686218493267, 'StockCode'),
 (0.011485879999222338, 'Description'),
 (0.003662549089237662, 'week1'),
 (0.0033253184625172466, 'Quantity'),
 (0.0018679069027254744, 'Friday'),
 (0.001557768384968039, 'week2'),
 (0.0007174881703392653, 'CustomerID'),
 (0.0006656599561963414, 'Thursday'),
 (0.0005580063464429727, 'Monday'),
 (0.0005085522812178416, 'A'),
 (0.0003915678039382759, '7'),
 (0.00010646998789522072, 'week3'),
 (0.00010144356266465782, 'C'),
 (8.291297782640816e-05, 'Morning'),
 (4.408637947187353e-05, '10'),
 (3.5805253154672064e-05, '14'),
 (1.4767208743299514e-05, '23'),
 (9.752663632259977e-06, 'Wednesday'),
 (9.337239436382329e-06, 'B'),
 (7.021919112946822e-06, 'Sunday'),
 (6.8565760405103035e-06, '0'),
 (6.049106753985864e-06, '2011'),
 (5.8387594344583015e-06, 'Tuesday'),
 (4.817039941555607e-06, 'Noon'),
 (3.768570073309043e-06, '19'),
 (1.1890560868356314e-06, '13'),
 (1.173757134211218e-06, '3'),
 (9.094579310944897e-07, '2010'),

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

rfr = rf_random.best_estimator_

forest_scores = cross_val_score(rfr,X,y,
                                scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [ 2.44308182  3.31446037 10.3959228   2.85348327 60.676986  ]
Mean: 15.936786853677194
Standard deviation: 22.560862694296908


Ridge Regressor

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1, solver="sag", random_state=42)


ridge_reg.fit(X_train,y_train)

predictions = ridge_reg.predict(X_test)

ridge_mse = mean_squared_error(y_test, predictions)
ridge_rmse = np.sqrt(ridge_mse)
ridge_rmse

18.121858045521286

SGD Regressor

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(penalty="l2", max_iter=1000, tol=1e-3, random_state=42)

sgd_reg.fit(X_train,y_train)

predictions = sgd_reg.predict(X_test)

sgd_mse = mean_squared_error(y_test, predictions)
sgd_rmse = np.sqrt(sgd_mse)
sgd_rmse

3340375.9961765218

Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)

lasso_reg.fit(X_train,y_train)

predictions = lasso_reg.predict(X_test)

lasso_mse = mean_squared_error(y_test, predictions)
lasso_rmse = np.sqrt(lasso_mse)
lasso_rmse

18.064142245272322

Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet
elastic_reg = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)

elastic_reg.fit(X_train,y_train)

predictions = elastic_reg.predict(X_test)

Elastic_mse = mean_squared_error(y_test, predictions)
Elastic_rmse = np.sqrt(Elastic_mse)
Elastic_rmse

18.069763023572886

Decision tree

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor( max_depth=4,
 max_features=None, max_leaf_nodes=None,
 min_impurity_split=1e-07, min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 presort=False, random_state=None)

tree_reg.fit(X_train,y_train)

predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

17.408599425771413

In [ ]:
tree_reg = DecisionTreeRegressor( max_depth=4,
max_features='auto', max_leaf_nodes=None,
min_impurity_split=1e-07, min_samples_leaf=1,
min_samples_split=2, min_weight_fraction_leaf=0.0,
presort=False, random_state=None)
tree_reg.fit(X_train,y_train)
predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

8.068861192879995


In [ ]:
min_samples_leafs = np.linspace(0.1, 1, 5, endpoint=True)

for min_samples_leaf in min_samples_leafs:
   tree_reg = DecisionTreeRegressor( max_depth=4,
   max_features='auto', max_leaf_nodes=None,
   min_impurity_split=1e-07, min_samples_leaf=1,
   min_samples_split=2, min_weight_fraction_leaf=0.0,
   presort=False, random_state=None)
   tree_reg.fit(X_train,y_train)
   predictions = tree_reg.predict(X_test)
   tree_mse = mean_squared_error(y_test, predictions)
   tree_rmse = np.sqrt(tree_mse)
   print(min_samples_leaf,tree_rmse)

0.1 8.068861192879995
0.325 8.068861192879995
0.55 8.068861192879995
0.775 8.068861192879995
1.0 8.068861192879995


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = rf_random.best_estimator_

rfr.fit(X_train,y_train)

predictions = rfr.predict(X_test)
rfr_mse = mean_squared_error(y_test, predictions)
rfr_rmse = np.sqrt(rfr_mse)
rfr_rmse

21.390881663977012

Support vector regression

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel="linear",C=1.0, epsilon=0.2)

svr.fit(X_train,y_train)

predictions = svr.predict(X_test)
svr_mse = mean_squared_error(y_test, predictions)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

18.134741108758785

KNN Regressor

In [ ]:
# Using K_Nearest neighbour regressor
# running for different K values to know which yields the max accuracy.
from sklearn.neighbors import KNeighborsRegressor
score = []
for k in range(1,20):    
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("At K = {}  Score is {} & RMSE is {}".format(k,clf.score(X_train,y_train),np.sqrt(mean_squared_error(y_test,y_pred))))
    score.append(clf.score(X_train,y_train))
k_max = score.index(max(score))+1
print( "At K = {}, Max Accuracy = {}".format(k_max, max(score)*100))

At K = 1  Score is 0.9449301118872662 & RMSE is 20.538312635539928
At K = 2  Score is 0.9724650559436331 & RMSE is 19.90775299047464
At K = 3  Score is 0.9724650559436331 & RMSE is 19.121223818920033
At K = 4  Score is 0.9724650559436331 & RMSE is 18.920654217124167
At K = 5  Score is 0.9724650559436331 & RMSE is 18.70781777568593
At K = 6  Score is 0.9724650559436331 & RMSE is 18.54538495422302
At K = 7  Score is 0.9724650559436331 & RMSE is 19.06764617516058
At K = 8  Score is 0.9724650559436331 & RMSE is 18.980159064535105
At K = 9  Score is 0.9724650559436331 & RMSE is 18.844640149612484
At K = 10  Score is 0.9724650559436331 & RMSE is 18.76930481695501
At K = 11  Score is 0.9724650559436331 & RMSE is 18.6769650242605
At K = 12  Score is 0.9724650559436331 & RMSE is 18.59954010034351
At K = 13  Score is 0.9724650559436331 & RMSE is 18.540757653286118
At K = 14  Score is 0.9724650559436331 & RMSE is 18.494335500390896
At K = 15  Score is 0.9724650559436331 & RMSE is 18.4596654313155

HistGradientBoostingRegressor

In [ ]:
# To use this experimental feature, we need to explicitly ask for it:
>>> from sklearn.experimental import enable_hist_gradient_boosting  # noqa
>>> from sklearn.ensemble import HistGradientBoostingRegressor

est = HistGradientBoostingRegressor()

X_train = X_train.toarray()
X_test = X_test.toarray()

est.fit(X_train,y_train)

predictions = est.predict(X_test)
est_mse = mean_squared_error(y_test, predictions)
est_rmse = np.sqrt(est_mse)
est_rmse


18.590288133438765

In [ ]:
df2 = df.copy()

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


num_attribs = ['StockCode','Quantity','Description','CustomerID','Country']
cat_attribs = [ 'Weekday','Timelight','Year','Month','Day']

full_pipeline2 = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])


X2= df2[['StockCode', 'Quantity', 'Country','Year','Month','Day','Weekday','Timelight','Description','CustomerID']]
y2 = df2['UnitPrice']


X2 = full_pipeline2.fit_transform(X2)
print(X2.shape)

from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2, test_size=0.25, random_state=None)

rfr2 = RandomForestRegressor(n_estimators=100, random_state=56)

rfr2.fit(X_train2,y_train2)

predictions = rfr2.predict(X_test2)
rfr_mse = mean_squared_error(y_test2, predictions)
rfr_rmse = np.sqrt(rfr_mse)
print(rfr_rmse)

(39968, 22)
29.301466410962615


XG Boosting

In [ ]:
import xgboost as xgb

#data_dmatrix = xgb.DMatrix(data=X,label=y)

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 25, 
                          alpha = 10, n_estimators = 4)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[19:01:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 12.733228


In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

params['eval_metric'] = "rmse"
num_boost_round = 999

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[5, 10, 15, 20, 25, 30]

In [ ]:
from scipy.stats import randint
x = randint(low=1, high=200)
x

In [4]:
import numpy as np
a=np.array([1,2,3,4,5])
type(a)

numpy.ndarray

In [6]:
b = np.log(a)

In [7]:
np.exp(b)

array([1., 2., 3., 4., 5.])